## Read Data

In [1]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [2]:
df_reviews = getDF('reviews_Electronics_5.json.gz')
print(len(df_reviews))
df_reviews.head(5)

1689188


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [3]:
df_reviews = df_reviews[['reviewerID', 'asin', 'unixReviewTime', 'overall']]

In [4]:
# df_reviews = df_reviews.sample(frac=0.1, random_state=10)

Read category metadata

In [5]:
df_meta = getDF('meta_Electronics.json.gz')
print(len(df_meta))
df_meta.head(5)

498196


,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,0511189877,http://ecx.images-amazon.com/images/I/41HaAhbv...,The CLIKR-5 UR5U-8780L remote control is desig...,"[[Electronics, Accessories & Supplies, Audio &...",CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,NaN,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


In [6]:
df_meta = df_meta[['asin', 'title', 'price', 'brand', 'categories']]
df_meta['categories'] = df_meta['categories'].map(lambda x: x[-1][-1])

In [7]:
df_reviews = df_reviews.merge(df_meta, on='asin', how='left')

In [8]:
df_reviews['label'] = df_reviews['overall'].apply(lambda x: 1 if x >= 3 else 0)
df_reviews.head(10)

,reviewerID,asin,unixReviewTime,overall,title,price,brand,categories,label
0,A000715434M800HLCENK9,B000UYYZ0M,1400457600,1.0,Draper V Screen Manual Projection Screen - 70&...,95.18,Draper Inc,Projection Screens,0
1,A000715434M800HLCENK9,B001EHAI6Y,1400457600,5.0,Elite Screens VMAX/Manual wall and ceiling han...,21.43,Elite Screens,Mounting Kits,1
2,A000715434M800HLCENK9,B003AFONFU,1400457600,3.0,Mount-It! Universal Projector Wall Mount with ...,75.96,NaN,Projector Mounts,1
3,A000715434M800HLCENK9,B003ES5ZUU,1400457600,2.0,AmazonBasics High-Speed HDMI Cable - 15 Feet (...,11.99,NaN,HDMI Cables,0
4,A000715434M800HLCENK9,B00HMZG3YS,1400457600,5.0,GoPro Case for GoPro Hero 1/2/3/3+ and Accesso...,25.99,NaN,Camera Cases,1
5,A00101847G3FJTWYGNQA,B00C7NSIO8,1379548800,5.0,ASUS Wireless Router (RT-N10P),29.99,Asus,Routers,1
6,A00101847G3FJTWYGNQA,B00IBPLI48,1379548800,4.0,TP-LINK TL-PA6010KIT AV600 Powerline Adapter S...,72.88,TP-LINK,Powerline Network Adapters,1
7,A00101847G3FJTWYGNQA,B005F778JO,1384041600,5.0,Thermaltake Dr. Power II Automated Power Suppl...,30.74,Thermaltake,Power Supplies,1
8,A00101847G3FJTWYGNQA,B006T9B6R2,1385769600,5.0,Anker&reg; Uspeed USB 3.0 Card Reader 8-in-1 f...,8.99,NaN,Memory Card Readers,1
9,A00101847G3FJTWYGNQA,B009NB8WR0,1385769600,5.0,Samsung Electronics MZ-7PD128BW 840 Pro Series...,109.98,Samsung,Internal Solid State Drives,1


In [12]:
temp

,reviewerID,asin,unixReviewTime,overall,title,price,brand,categories,label
0,A000715434M800HLCENK9,B000UYYZ0M,1400457600,1.0,Draper V Screen Manual Projection Screen - 70&...,95.18,Draper Inc,Projection Screens,0
1,A000715434M800HLCENK9,B001EHAI6Y,1400457600,5.0,Elite Screens VMAX/Manual wall and ceiling han...,21.43,Elite Screens,Mounting Kits,1
2,A000715434M800HLCENK9,B003AFONFU,1400457600,3.0,Mount-It! Universal Projector Wall Mount with ...,75.96,NaN,Projector Mounts,1
3,A000715434M800HLCENK9,B003ES5ZUU,1400457600,2.0,AmazonBasics High-Speed HDMI Cable - 15 Feet (...,11.99,NaN,HDMI Cables,0
4,A000715434M800HLCENK9,B00HMZG3YS,1400457600,5.0,GoPro Case for GoPro Hero 1/2/3/3+ and Accesso...,25.99,NaN,Camera Cases,1
5,A00101847G3FJTWYGNQA,B00C7NSIO8,1379548800,5.0,ASUS Wireless Router (RT-N10P),29.99,Asus,Routers,1
6,A00101847G3FJTWYGNQA,B00IBPLI48,1379548800,4.0,TP-LINK TL-PA6010KIT AV600 Powerline Adapter S...,72.88,TP-LINK,Powerline Network Adapters,1
7,A00101847G3FJTWYGNQA,B005F778JO,1384041600,5.0,Thermaltake Dr. Power II Automated Power Suppl...,30.74,Thermaltake,Power Supplies,1
8,A00101847G3FJTWYGNQA,B006T9B6R2,1385769600,5.0,Anker&reg; Uspeed USB 3.0 Card Reader 8-in-1 f...,8.99,NaN,Memory Card Readers,1
9,A00101847G3FJTWYGNQA,B009NB8WR0,1385769600,5.0,Samsung Electronics MZ-7PD128BW 840 Pro Series...,109.98,Samsung,Internal Solid State Drives,1


In [23]:
df_reviews = df_reviews.sort_values(by=['reviewerID', 'unixReviewTime']).reset_index(drop=True)

In [24]:
from copy import deepcopy

def get_user_seqs(df: pd.DataFrame):
    user_seqs = []
    cur_seq = []
    for idx, r in df.iterrows():
        user_seqs.append(deepcopy(cur_seq))
        if r['label'] > 0:
            cur_seq.append(r['asin'])

    df['his_asin_seq'] = user_seqs
    return df

df_reviews = df_reviews.groupby('reviewerID', group_keys=False).apply(lambda x: get_user_seqs(x))

In [26]:
df_reviews.head()

,reviewerID,asin,unixReviewTime,overall,title,price,brand,categories,label,his_asin_seq
0,A000715434M800HLCENK9,B000UYYZ0M,1400457600,1.0,Draper V Screen Manual Projection Screen - 70&...,95.18,Draper Inc,Projection Screens,0,[]
1,A000715434M800HLCENK9,B001EHAI6Y,1400457600,5.0,Elite Screens VMAX/Manual wall and ceiling han...,21.43,Elite Screens,Mounting Kits,1,[]
2,A000715434M800HLCENK9,B003AFONFU,1400457600,3.0,Mount-It! Universal Projector Wall Mount with ...,75.96,NaN,Projector Mounts,1,[B001EHAI6Y]
3,A000715434M800HLCENK9,B003ES5ZUU,1400457600,2.0,AmazonBasics High-Speed HDMI Cable - 15 Feet (...,11.99,NaN,HDMI Cables,0,"[B001EHAI6Y, B003AFONFU]"
4,A000715434M800HLCENK9,B00HMZG3YS,1400457600,5.0,GoPro Case for GoPro Hero 1/2/3/3+ and Accesso...,25.99,NaN,Camera Cases,1,"[B001EHAI6Y, B003AFONFU]"


In [ ]:
print(len(df_reviews))

1689188


In [27]:
import joblib

joblib.dump(df_reviews, 'amazon_data.pkl')

['amazon_data.pkl']